In [1]:
# 安裝需要的套件
!pip install tensorflow

import tensorflow as tf
import numpy as np
import os
import zipfile
from pathlib import Path

# 下載資料集
!wget --no-check-certificate 'https://storage.googleapis.com/learning-datasets/rps.zip' -O /tmp/rps.zip
!wget --no-check-certificate 'https://storage.googleapis.com/learning-datasets/rps-test-set.zip' -O /tmp/rps-test-set.zip

# 解壓縮資料集
!unzip -q /tmp/rps.zip -d /tmp/
!unzip -q /tmp/rps-test-set.zip -d /tmp/

# 檢查資料夾結構
!ls /tmp
!echo "訓練資料夾內容："
!ls /tmp/rps
!echo "測試資料夾內容："
!ls /tmp/rps-test-set

--2024-12-13 12:28:28--  https://storage.googleapis.com/learning-datasets/rps.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.188.207, 172.253.117.207, 173.194.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 200682221 (191M) [application/zip]
Saving to: ‘/tmp/rps.zip’

/tmp/rps.zip        100%[===================>] 191.38M   167MB/s    in 1.1s    

2024-12-13 12:28:30 (167 MB/s) - ‘/tmp/rps.zip’ saved [200682221/200682221]

--2024-12-13 12:28:30--  https://storage.googleapis.com/learning-datasets/rps-test-set.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.188.207, 172.253.117.207, 173.194.203.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.188.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29516758 (28M) [application/zip]
Saving to: ‘/tmp/rps-test-set.zip’

/t

In [2]:
# 設定參數
BATCH_SIZE = 32
IMG_SIZE = (160, 160)

# 載入資料集
train_dataset = tf.keras.utils.image_dataset_from_directory(
    '/tmp/rps',  # 修正的路徑
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    '/tmp/rps',  # 修正的路徑
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    '/tmp/rps-test-set',  # 修正的路徑
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 2520 files belonging to 3 classes.
Using 2016 files for training.
Found 2520 files belonging to 3 classes.
Using 504 files for validation.
Found 372 files belonging to 3 classes.


In [5]:
# 資料預處理和增強
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=AUTOTUNE)

# 資料增強層
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

In [8]:
# 建立模型
model = tf.keras.Sequential([
    # 指定輸入形狀
    tf.keras.layers.Input(shape=(160, 160, 3)),  # 加入這行，指定輸入維度

    # 資料增強層
    data_augmentation,

    # 其他層
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(3)
])

# 編譯模型
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# 顯示模型結構
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)            │ (None, 160, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ rescaling_3 (Rescaling)              │ (None, 160, 160, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 158, 158, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ (None, 79, 79, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 77, 77, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ (None, 38, 38, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 36, 36, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ (None, 18, 18, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 18, 18, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ (None, 20736)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 128)                 │       2,654,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,711,043 (10.34 MB)

 Trainable params: 2,711,043 (10.34 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# 訓練模型
history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=10
)

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 120s 2s/step - accuracy: 0.4170 - loss: 1.1468 - val_accuracy: 0.8095 - val_loss: 0.4643
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step - accuracy: 0.8359 - loss: 0.4089 - val_accuracy: 0.9206 - val_loss: 0.2320
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 108s 2s/step - accuracy: 0.9296 - loss: 0.1966 - val_accuracy: 0.9385 - val_loss: 0.1487
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9475 - loss: 0.1579 - val_accuracy: 0.9544 - val_loss: 0.1474
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 145s 2s/step - accuracy: 0.9635 - loss: 0.1147 - val_accuracy: 0.9841 - val_loss: 0.0732
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9748 - loss: 0.0859 - val_accuracy: 0.9821 - val_loss: 0.0548
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 109s 2s/step - accuracy: 0.9764 - loss: 0.0731 - val_accuracy: 0.9861 - val_loss: 0.0558
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 114s 2s/step - accuracy: 0.9817 - loss: 0.0528 - val_accuracy: 0.9722 - v

In [11]:
# 評估模型
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"\nTest accuracy: {test_accuracy:.2f}")

12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 463ms/step - accuracy: 0.9325 - loss: 0.2412

Test accuracy: 0.92


In [12]:
# 預測函數
def predict_image(image_path):
    img = tf.keras.utils.load_img(
        image_path,
        target_size=IMG_SIZE
    )
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)
    score = tf.nn.softmax(predictions[0])

    class_names = ['paper', 'rock', 'scissors']
    predicted_class = class_names[np.argmax(score)]
    confidence = 100 * np.max(score)

    return predicted_class, confidence

# 測試預測
test_image = '/tmp/rps-test-set/rock/testrock01-00.png'  # 修正的路徑
predicted_class, confidence = predict_image(test_image)
print(f"This image most likely belongs to {predicted_class} with a {confidence:.2f} percent confidence.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
This image most likely belongs to rock with a 100.00 percent confidence.
